In [1]:
import os 
from pprint import pprint

Check that output from other examples has been generated 

In [2]:
from pathlib2 import Path

In [3]:
need_files = ['methane.xyz']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run structures.ipynb")
        os.system("jupyter nbconvert --to python  structures.ipynb")
        os.system("python structures.py")

Need to run structures.ipynb


In [4]:
need_files = ['ethane.xyz']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run buildingblocks.ipynb")
        os.system("jupyter nbconvert --to python  buildingblocks.ipynb")
        os.system("python buildingblocks.py")

Need to run buildingblocks.ipynb


In [5]:
import streamm

Now let's create project and resource to keep track of our work

In [6]:
nwchem_example = streamm.Project('nwchem_example')
res_local = streamm.Resource('local')

In [7]:
res_local.make_dir()

Update relative location of templates directory 

In [8]:
res_local.dir['templates'] =  os.path.join(res_local.dir['home'],'..','templates','')

Make sure this is the location of the templates directory that comes with the streamm git repository  https://github.com/NREL/streamm-tools

In [9]:
print res_local.dir['templates']

/Users/tkemper/Development/streamm-tools/examples/../templates/


Create the local directories that will store our files 

In [10]:
nwchem_example.make_dir()

Tell the project about our directories 

In [11]:
nwchem_example.set_resource(res_local)

Read in the methane.xyz file created in the structure.ipynb example 

In [12]:
methane = streamm.Buildingblock('methane')

In [13]:
methane.read_xyz()

Create the neighbor list 

In [14]:
methane.bonded_nblist = methane.guess_nblist(0,radii_buffer=1.25)

and the bonded interactions

In [15]:
methane.bonded_bonds()
methane.bonded_angles()
methane.bonded_dih()

In [16]:
print methane.n_particles

5


In [17]:
print methane.print_properties()

 n_particles:5 
 n_bonds:4
 n_angles:6
 n_dihedrals:0
 n_impropers:0


Set the paramkeys so we can identify force field parameters later on

In [18]:
for pkey,p in methane.particles.iteritems():
    if( p.symbol == 'C' ):
        p.paramkey = 'CT'
    elif( p.symbol == 'H' ):
        p.paramkey = 'HC'

In [19]:
for pk,p in methane.particles.iteritems():
    p.residue = 1
    p.resname = 'METH'

Set some rsites to be able to join molecules together 

In [20]:
methane.particles[1].rsite = 'RH'
methane.particles[2].rsite = 'RH'

In [21]:
methane.find_rsites()

In [22]:
print methane.show_rsites()

rsite:RH[ paticle:atom[1] H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom[2] H (H) index:2 n_bonds:1] 



Read in ethane.xyz from the buildinblock.ipynb example 

In [23]:
ethane = streamm.Buildingblock('ethane')

In [24]:
ethane.read_xyz()

Guess bonded neighbor list based on `bonded_radii`

In [25]:
ethane.bonded_nblist = ethane.guess_nblist(0,radii_buffer=1.25)

In [26]:
ethane.bonded_bonds()
ethane.bonded_angles()
ethane.bonded_dih()

In [27]:
print ethane.print_properties()

 n_particles:8 
 n_bonds:7
 n_angles:12
 n_dihedrals:9
 n_impropers:0


Set the `paramkey`'s  as described in the force field example

In [28]:
for pkey,p in ethane.particles.iteritems():
    if( p.symbol == 'C' ):
        p.paramkey = 'CT'
    elif( p.symbol == 'H' ):
        p.paramkey = 'HC'

Set the `resname` of each particle to `ETH` 

In [29]:
for pk,p in ethane.particles.iteritems():
    p.residue = 1
    p.resname = 'ETH'

Set `rsite`'s to hydrogens to be replaced during join

In [30]:
ethane.particles[1].rsite = 'RH'
ethane.particles[5].rsite = 'RH'

Run `find_rsites()` to populate `func` list

In [31]:
ethane.find_rsites()

In [32]:
print ethane.show_rsites()

rsite:RH[ paticle:atom[1] H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom[5] H (H) index:5 n_bonds:1] 



In [33]:
import copy

Create octane from ethane 

Copy ethane to a new Buildingblock octane 

In [34]:
octane = copy.deepcopy(ethane)

In [35]:
from streamm.structures.buildingblock import attach

Then attach 3 more ethanes to make an octane 

In [68]:
for i in range(3):
    octane = attach(octane,ethane,'RH',1,'RH',0)

Update the tag

In [69]:
octane.tag = 'octane'

Rename the residue and resname for octane

In [70]:
for pk,p in octane.particles.iteritems():
    p.residue = 2
    p.resname = "OCT"
 

In [71]:
octane.write_xyz()

Print new `rsite`'s 

In [73]:
print octane.show_rsites()

rsite:RH[ paticle:atom[1] H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom[95] H (H) index:95 n_bonds:1] 
rsite:R2[ paticle:atom[14] H (H) index:14 n_bonds:1] 



Find the 4th carbon to attach an ethane 

In [74]:
print octane.particles[14].symbol

H


In [75]:
octane.particles[14].rsite = 'R2'

In [76]:
octane.find_rsites()

Attach the ethane to the fourth carbon to make 4-ethyloctane

In [77]:
ethyl_octane = attach(octane,ethane,'R2',0,'RH',0)

In [78]:
ethyl_octane.tag = '4-ethyloctane'

In [79]:
ethyl_octane.write_xyz()

Create NWChem Calculation object 

In [80]:
nwchem_ethyl_octane = streamm.NWChem('nw_ethyl_octane_OPT')

Add calculation to project

In [81]:
nwchem_example.add_calc(nwchem_ethyl_octane)

Set the structure of the calculation to 4-ethyloctane

In [82]:
nwchem_ethyl_octane.strucC = ethyl_octane

Set the resource to be local

In [83]:
nwchem_ethyl_octane.set_resource(res_local)

Make the local directories 

In [84]:
nwchem_ethyl_octane.make_dir()

Change to the `scratch` directory

In [85]:
os.chdir(nwchem_ethyl_octane.dir['scratch'])

Copy the template files to the scratch direcotry 

In [86]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
nwchem_ethyl_octane.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [87]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'scratch'
nwchem_ethyl_octane.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

Read in the template files and add them to the `str` dictionary

In [88]:
nwchem_ethyl_octane.load_str('templates','nw')        
nwchem_ethyl_octane.load_str('templates','run')

Set the properties dictionary to desired calculation details

In [89]:
nwchem_ethyl_octane.properties['basis'] = '6-31g'
nwchem_ethyl_octane.properties['method'] = 'UHF'
nwchem_ethyl_octane.properties['charge'] = 0
nwchem_ethyl_octane.properties['spin_mult'] = 1
nwchem_ethyl_octane.properties['task'] = 'SCF optimize'
nwchem_ethyl_octane.properties['coord'] = nwchem_ethyl_octane.strucC.write_coord()

In [90]:
pprint(nwchem_ethyl_octane.properties)

{u'allocation': u'',
 'basis': '6-31g',
 'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -0.00000001       3.30826383       2.86504051 \n     H       0.39999998       4.41924795       3.07884950 \n     H      -1.20000001       3.30826383       2.86504051 \n     H       0.39999998       2.56760776       3.72027648 \n     C       0.44666666       2.89473085       1.67127363 \n     H       1.64666665       2.89473085       1.67127363 \n     H       0.04666666       3.63538692       0.81603766 \n     C      -0.00000001       1.65413192       1.43252025 \n     H      -1.20000000       1.65413192       1.43252025 \n     H       0.39999999       0.91347585       2.28775623 \n     C       0.44666666       1.24059894       0.23875338 \n     H       1.64666665       1.24059894       0.23875338 \n     H       0.04666667       1.98125500      -0.61648260 \n     C       0.00000000       0.00000000       0.00000000 \n     H       0.

Replace the keys in the template strings and write the input files 

In [91]:
nwchem_ethyl_octane.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_ethyl_octane.tag))

Add the input file to the properties to be written into the run file

In [92]:
nwchem_ethyl_octane.properties['input_nw'] = nwchem_ethyl_octane.files['input']['nw']
nwchem_ethyl_octane.replacewrite_prop('run','scripts','run','%s.sh'%(nwchem_ethyl_octane.tag))

Add the log file to the files dictionary 

In [93]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_ethyl_octane.tag)
nwchem_ethyl_octane.add_file(file_type,file_key,file_name)

Change back to the root directory and write a json file

In [94]:
os.chdir(nwchem_example.dir['home'])
nwchem_example.dump_json()

Change back to scratch

In [95]:
os.chdir(nwchem_example.dir['scratch'])

Run the bash script for the calculation or submit the job to the cluster

In [63]:
nwchem_ethyl_octane.run()

Check the status of all the calculations in the project

In [64]:
nwchem_example.check()

Calculation nw_ethyl_octane_OPT has status written


Run the analysis

In [65]:
nwchem_example.analysis()

Tar and zip the results and copy them to a storage location

In [66]:
nwchem_example.store()

Save json in home directory

In [67]:
os.chdir(nwchem_example.dir['home'])
nwchem_example.dump_json()